In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# This will collect the links for all cities on craigslist. We will have to sparse through
# these and cut out the non-US cities.
def city_link_collector():
    # this is the craigslist page with every city
    main_page = requests.get('https://www.craigslist.org/about/sites')
    soup = BeautifulSoup(main_page.text, 'html.parser')
    
    all_list = []
    uscity_list = []
    city_list = []
    
    for i in range(4):
        for box in soup.find_all('div', class_='box box_{}'.format(i+1)):
            all_list.append(box.find_all('a'))
    for _ in all_list[:20:7]:
        for __ in _:
            uscity_list.append(__)
    for ___ in all_list[20][0:94]:
        uscity_list.append(___)
    
    for idx, city in enumerate(uscity_list):
        city_list.append(str(uscity_list[idx]).split('''"''')[1])
    
    '''These listings are all a subset of Miami and break the scraper if entered in this format'''
    city_list.remove('http://miami.craigslist.org/brw/')
    city_list.remove('http://miami.craigslist.org/mdc/')
    city_list.remove('http://miami.craigslist.org/pbc/')
    '''Here is the Miami list to capture the dropped entries and will not break the scraper'''
    city_list.append('http://miami.craigslist.org/')
    return city_list
    #                     #posting date
    #                     #grab the datetime element 0 for date and 1 for time
    #                     post_datetime = box.find('time', class_= 'result-date')['datetime']
    #                     post_timing.append(post_datetime)

            

In [3]:
city_list = city_link_collector()
len(city_list)

415

In [85]:
city_list[74]

'https://jacksonville.craigslist.org/'

In [121]:
df = pd.read_csv('motorcycle_toy_df')

In [123]:
df['title'][0]

'<span id="titletextonly">2016 Honda CBR300r</span>'

In [81]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0.1', axis=1, inplace=True)
df.drop('Unnamed: 0.1.1', axis=1, inplace=True)
df.index.names = ['index']
df.replace(df['price'], df['price'].to_string(), inplace=True)
df = df.replace(np.nan, '', regex=True)

In [82]:
for _ in range(len(df)):
    df.replace(df['title'][_], str(df['title'][_]).replace('''<span id="titletextonly">''', '').replace('''</span>''', ''), inplace=True)
    df.replace(df['price'][_], str((df['price'][_].replace('''</span>''', '').replace('''<span class="price">$''', ''))), inplace=True)
    df.replace(df['neighborhood'][_], str(df['neighborhood'][_].replace('''<small> (''', '').replace(''')</small>''', '')), inplace=True)
    df.replace(df['attributes'][_], str(df['attributes'][_].replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''', '').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '').replace('''<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]''', '')), inplace=True )
    df.replace(df['description'][_], str((df['description'][_].replace('''<section id="postingbody">\n<div class="print-information print-qrcode-container">\n<p class="print-qrcode-label">QR Code Link to This Post</p>\n<div class="print-qrcode" data-location=''', '').replace('''></div>\n</div>''', '').replace('''\n''', ' ').replace('''<br/>''', '').replace('''</section>''', '').split('''.html" '''))[-1]), inplace=True)

In [83]:
df = df.replace('', np.nan, regex=True)
df['price'] = pd.to_numeric(df['price'])
df.head()

,title,price,neighborhood,attributes,description,city
index,,,,,,
0,2016 Honda CBR300r,2900.0,Auburn,"2016 honda cbr300r,condition:excellent,engine ...","2016 Honda CBR300r 6,186 miles as of listing a...",https://auburn.craigslist.org/
1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",NaN,FAYETTEVILLE,"all bikes,condition:excellent,fuel:gas,title ...","WE SHIP NATIONWIDE, FINANCE NATIONWIDE! YOU SE...",https://auburn.craigslist.org/
2,2017 Yamaha FZ 07 very low miles,5000.0,Near Montgomery,"2017 Yamaha FZ-07,condition:like new,engine di...","2017 Yamaha FZ-07 with only 1100 miles, bike i...",https://auburn.craigslist.org/
3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,NaN,"<small>\n (<a href=""https://www.google....","2010 Honda GL1800 CSC,condition:like new,engin...","2010 Yellow GL1800 CSC Viper LOOK!!! only 8,2...",https://auburn.craigslist.org/
4,2006 Honda XR650L *Price Drop*,3800.0,Auburn,"2006 honda xr650l,condition:excellent,engine d...","2006 Honda XR650L 8,000 miles. This number w...",https://auburn.craigslist.org/


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14103 entries, 0 to 14102
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         14102 non-null  object 
 1   price         13591 non-null  float64
 2   neighborhood  14033 non-null  object 
 3   attributes    14103 non-null  object 
 4   description   14102 non-null  object 
 5   city          14103 non-null  object 
dtypes: float64(1), object(5)
memory usage: 661.2+ KB


In [78]:
# quicklst = []
# for _ in range(len(df.attributes)):
#     quicklst.append((len(df['attributes'][_].split(','))))
#     for idx, i in enumerate(quicklst):
#         if i ==13:
#             print(idx)

In [44]:
df['attributes'][2969].split(',')

['2014 2014 Aprilia Dorsoduro ABS',
 'VIN:ZD4SMC010ES000023',
 'condition:like new',
 'cryptocurrency ok',
 'delivery available',
 'engine displacement (CC):750',
 'fuel:gas',
 'odometer:17378',
 'paint color:white',
 'street legal',
 'title status:clean',
 'transmission:manual',
 'type:dual-sport<p class="attrgroup">\n<span class="otherpostings">\n<a href="//inlandempire.craigslist.org/search/sss?userid=319057509">\nmore ads  by this user        </a>\n</span>\n</p>]']

In [120]:
# def attributes_in_this_mf():

model = []
vin = []
condition = []
cryptocurrency = []
delivery = []
engine_size = []
fuel_type = []
mileage = []
paint_color = []
street_legal = []
title_status = []
transmission = []
bike_type = []

vin_ = 'VIN:'
condition_ = 'condition:'
cryptocurrency_ = 'cryptocurrency'
delivery_ = 'delivery'
engine_size_ = 'engine displacement'
fuel_type_ = 'fuel:'
mileage_ = 'odometer:'
paint_color_ = 'paint color:'
street_legal_ = 'street legal'
title_status_ = 'title status:'
transmission_ = 'transmission:'
bike_type_ = 'type:'

for idx, _ in enumerate(df['attributes'][0:50]):
    bike_attributes = (df['attributes'][idx].split(','))
    
    vin_res = [i for i in bike_attributes if vin_ in i]
    condition_res = [i for i in bike_attributes if condition_ in i]
    cryptocurrency_res = [i for i in bike_attributes if cryptocurrency_ in i]
    delivery_res = [i for i in bike_attributes if delivery_ in i]
    engine_size_res = [i for i in bike_attributes if engine_size_ in i]
    fuel_type_res = [i for i in bike_attributes if fuel_type_ in i]
    mileage_res = [i for i in bike_attributes if mileage_ in i]
    paint_color_res = [i for i in bike_attributes if paint_color_ in i]
    street_legal_res = [i for i in bike_attributes if street_legal_ in i]
    title_status_res = [i for i in bike_attributes if title_status_ in i]
    transmission_res = [i for i in bike_attributes if transmission_ in i]
#     transmission_res = transmission_res.rstrip('<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]')
    bike_type_res = str(i for i in bike_attributes if bike_type_ in i)
    
    model.append(bike_attributes[0])
    vin.append(vin_res)
    condition.append(condition_res)
    cryptocurrency.append((cryptocurrency_res))
    delivery.append(delivery_res)
    engine_size.append(engine_size_res)
    fuel_type.append(fuel_type_res)
    mileage.append(mileage_res)
    paint_color.append(paint_color_res)
    street_legal.append(street_legal_res)
    title_status.append(title_status_res)
    transmission.append(transmission_res)
    bike_type.append(bike_type_res)

# df['model'] = np.array(model)
# df['vin'] = np.array(vin)
# df['condition'] = np.array(condition)
# df['cryptocurrency'] = np.array(cryptocurrency)
# df['delivery'] = np.array(delivery)
# df['engine_size'] = np.array(engine_size)
# df['fuel_type'] = np.array(fuel_type)
# df['mileage'] = np.array(mileage)
# df['paint_color'] = np.array(paint_color)
# df['street_legal'] = np.array(street_legal)
# df['title_status'] = np.array(title_status)
# df['transmission'] = np.array(transmission)
# df['bike_type'] = np.array(bike_type)

# df = df.replace('', np.nan, regex=True)
# df['vin'] = pd.to_numeric(df['vin'])
# df['mileage'] = pd.to_numeric(df['mileage'])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['VIN:1HD1MAL16EB851071']
['VIN:1HD1MAL16EB851071']
['VIN:WB1061203EZZ25082']
['VIN:JS1GR7MA6H2101909']
[]
['VIN:56KMSA008K3140133']
[]
[]
['VIN:RLHKF1803FY000831']
['VIN:1HD4NAA18FC503670']
[]
[]
['VIN:1HFSC0218DA326320']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['VIN:1HD1BWV14DB042926']
['VIN:1HD1BWV14DB042926']
[]


In [115]:
print(vin)

['<generator object <genexpr> at 0x7fd122cdcd50>', '<generator object <genexpr> at 0x7fd127790e50>', '<generator object <genexpr> at 0x7fd122c641d0>', '<generator object <genexpr> at 0x7fd122cdc550>', '<generator object <genexpr> at 0x7fd1269d4f50>', '<generator object <genexpr> at 0x7fd126971e50>', '<generator object <genexpr> at 0x7fd1269d4050>', '<generator object <genexpr> at 0x7fd1269d47d0>', '<generator object <genexpr> at 0x7fd1269d4850>', '<generator object <genexpr> at 0x7fd123175e50>', '<generator object <genexpr> at 0x7fd122cdcf50>', '<generator object <genexpr> at 0x7fd122cdcd50>', '<generator object <genexpr> at 0x7fd127790e50>', '<generator object <genexpr> at 0x7fd122c641d0>', '<generator object <genexpr> at 0x7fd122cdc550>', '<generator object <genexpr> at 0x7fd1269d4f50>', '<generator object <genexpr> at 0x7fd126971e50>', '<generator object <genexpr> at 0x7fd1282a77d0>', '<generator object <genexpr> at 0x7fd1282a7050>', '<generator object <genexpr> at 0x7fd1282a7850>',

In [98]:
model.append(bike_attributes[0])
vin.append(vin_res.lstrip('VIN:'))
condition.append(condition_res[0].lstrip('condition:'))
cryptocurrency.append((cryptocurrency_res[0]))
delivery.append(delivery_res[0])
engine_size.append(engine_size_res[0].lstrip('engine displacement (CC):'))
fuel_type.append(fuel_type_res[0].lstrip('fuel:'))
mileage.append(mileage_res[0].lstrip('odometer:'))
paint_color.append(paint_color_res[0].lstrip('paint color:'))
street_legal.append(street_legal_res[0])
title_status.append(title_status_res[0].lstrip('title status:'))
transmission.append(transmission_res[0].lstrip('transmission:'))
bike_type.append(bike_type_res[0].lstrip('type:'))


['2016 honda cbr300r', 'condition:excellent', 'engine displacement (CC):286', 'fuel:gas', 'paint color:yellow', 'street legal', 'title status:clean', 'transmission:manual', 'type:sport bike']


In [99]:
df['cryptocurrency'][0:50]

index
0     <generator object <genexpr> at 0x7fd122c641d0>
1     <generator object <genexpr> at 0x7fd122c641d0>
2     <generator object <genexpr> at 0x7fd122c641d0>
3     <generator object <genexpr> at 0x7fd122c641d0>
4     <generator object <genexpr> at 0x7fd122c641d0>
5     <generator object <genexpr> at 0x7fd122c641d0>
6     <generator object <genexpr> at 0x7fd122c641d0>
7     <generator object <genexpr> at 0x7fd122c641d0>
8     <generator object <genexpr> at 0x7fd122c641d0>
9     <generator object <genexpr> at 0x7fd126971b50>
10    <generator object <genexpr> at 0x7fd126971b50>
11    <generator object <genexpr> at 0x7fd126971b50>
12    <generator object <genexpr> at 0x7fd126971b50>
13    <generator object <genexpr> at 0x7fd126971b50>
14    <generator object <genexpr> at 0x7fd126971b50>
15    <generator object <genexpr> at 0x7fd126971b50>
16    <generator object <genexpr> at 0x7fd126971b50>
17    <generator object <genexpr> at 0x7fd122cdc550>
18    <generator object <genexpr> at 0x7

In [77]:
bike_type 

[['type:sport bike'],
 [],
 ['type:street'],
 ['type:trike<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=118773920">\nmore ads  by this user        </a>\n</span>\n</p>]'],
 ['type:dual-sport'],
 [],
 ['type:sport touring'],
 ['type:touring<p class="attrgroup">\n<span class="otherpostings">\n<a href="//bham.craigslist.org/search/sss?userid=138551021">\nmore ads  by this user        </a>\n</span>\n</p>]'],
 ['type:touring'],
 ['type:sport touring'],
 ['type:touring'],
 ['type:cruiser'],
 ['type:touring'],
 ['type:trike'],
 [],
 ['type:touring'],
 ['type:touring'],
 [],
 ['type:sport bike'],
 ['type:trike<p class="attrgroup">\n<span class="otherpostings">\n<a href="//bham.craigslist.org/search/sss?userid=47732091">\nmore ads  by this user        </a>\n</span>\n</p>]'],
 ['type:trike<p class="attrgroup">\n<span class="otherpostings">\n<a href="//huntsville.craigslist.org/search/sss?userid=47732091">\nmore ads  by this user        </a

In [35]:

df['attributes'][0].replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''', '').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '')

'2016 honda cbr300r,condition:excellent,engine displacement (CC):286,fuel:gas,paint color:yellow,street legal,title status:clean,transmission:manual,type:sport bike'

In [45]:
df.replace(df['attributes'][0], df['attributes'][0].lstrip('''</b></span>\n<br/>\n</p>'''), inplace=True)
df['attributes'][1]

'[<p class="attrgroup">\n<span><b> all bikes</b></span>\n<br/>\n</p>, <p class="attrgroup">\n<span>condition: <b>excellent</b></span>\n<br/>\n<span>fuel: <b>gas</b></span>\n<br/>\n<span>title status: <b>clean</b></span>\n<br/>\n<span>transmission: <b>manual</b></span>\n<br/>\n</p>, <p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]'

In [53]:
(df['description'][0].replace('''<section id="postingbody">\n<div class="print-information print-qrcode-container">\n<p class="print-qrcode-label">QR Code Link to This Post</p>\n<div class="print-qrcode" data-location=''', '').replace('''></div>\n</div>''', '').replace('''\n''', ' ').replace('''<br/>''', '').replace('''</section>''', '').split('''.html" '''))[-1]

'2016 Honda CBR300r 6,186 miles as of listing after market: fender eliminator kit and bubble windscreen I have the original parts if you would prefer those. clean title  This is an excellent starter bike or daily commuter. 50+ mpg around auburn and the best parking on campus!  Not looking to trade  '

In [16]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,price,neighborhood,attributes,description,city
0,0,0,0,2016 Honda CBR300r,"<span class=""price"">$2900</span>",Auburn,"[<p class=""attrgroup"">\n<span><b>2016 honda cb...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
1,1,1,1,"BAD CREDIT, NO CREDIT, OK! WE WORK WITH EVERYONE!",,<small> (FAYETTEVILLE)</small>,"[<p class=""attrgroup"">\n<span><b> all bikes</b...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
2,2,2,2,2017 Yamaha FZ 07 very low mile,"<span class=""price"">$5000</span>",<small> (Near Montgomery)</small>,"[<p class=""attrgroup"">\n<span><b>2017 Yamaha F...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
3,3,3,3,2010 Yellow Goldwing GL1800 CSC(see pics/price...,,"<small>\n (<a href=""https://www.google....","[<p class=""attrgroup"">\n<span><b>2010 Honda GL...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
4,4,4,4,2006 Honda XR650L *Price Drop*,"<span class=""price"">$3800</span>",Auburn,"[<p class=""attrgroup"">\n<span><b>2006 honda xr...","<section id=""postingbody"">\n<div class=""print-...",https://auburn.craigslist.org/
